In [1]:
from docplex.cp.model import *
from docplex.cp.config import get_default

In [2]:
from config import setup
setup()

In [3]:
n = 4

F = ['f1', 'f2', 'f3', 'f4']
H = ['h1', 'h2', 'h3', 'h4']

L = {
    "f1":['h2', 'h1', 'h3', 'h4'],
    "f2":['h3', 'h4', 'h1', 'h2'],
    "f3":['h1', 'h3', 'h4', 'h2'],
    "f4":['h2', 'h1', 'h3', 'h4'],

    "h1":['f2', 'f3', 'f1', 'f4'],
    "h2":['f4', 'f1', 'f3', 'f2'],
    "h3":['f2', 'f4', 'f1', 'f3'],
    "h4":['f3', 'f1', 'f4', 'f2']
}

mdl = CpoModel(name="Mariages")

In [4]:
#Lpos[h,f] postition de f dans L(h) + position de h dans L(h)
LPos = {}
for h in H:
    for f in F:
        LPos[h,f]=L[h].index(f)+L[f].index(h)
LPos

{('h1', 'f1'): 3,
 ('h1', 'f2'): 2,
 ('h1', 'f3'): 1,
 ('h1', 'f4'): 4,
 ('h2', 'f1'): 1,
 ('h2', 'f2'): 6,
 ('h2', 'f3'): 5,
 ('h2', 'f4'): 0,
 ('h3', 'f1'): 4,
 ('h3', 'f2'): 0,
 ('h3', 'f3'): 4,
 ('h3', 'f4'): 3,
 ('h4', 'f1'): 4,
 ('h4', 'f2'): 4,
 ('h4', 'f3'): 2,
 ('h4', 'f4'): 5}

In [5]:
#x[h,f] mariage entre h et f
x = {}
for h in H:
    for f in F:
        x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)

#y[i] cout du marriage de i (somme des couts du mariage pour les 2 partis) 
y = {}
for h in H:
    y[h]=mdl.integer_var(0,2*(n-1),name="y_"+h)



In [6]:
#Ajout des contraintes

#Contrainte de monogamie
for h in H:
    mdl.add(sum([x[h,f] for f in F])==1)
for f in F:
    mdl.add(sum([x[h,f] for h in H])==1)


#Valeur de y:
for h in H:
    mdl.add(y[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))

#On cherches à minimiser les couts des marriages
mdl.add(minimize(sum([y[h] for h in H])))

In [7]:
solution = mdl.solve()
solution.print_solution()

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 20 variables, 12 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 27.2 (before), 27.2 (after)
 !  . Memory usage      : 300.4 kB (before), 300.4 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         20                 -
 + New bound is 0
                        0         20                 -
 + New bound is 4
                        8         11        F     0 != x_h1_f4
 + New bound is 5
 *            16       14  0.00s               (gap is 68.75%)
 *             9       17  0.00s               (gap is 44.44%)
 *             5       24  0.00s               (gap is 0.00%)
 ! ------------------------------

In [8]:
#Affichage des de la solution:
for h in H:
    for f in F:
        if(solution[x[h,f]]==1):
            print("Mariage entre ",h," et ",f," (Cout = ",LPos[h,f],")")


Mariage entre  h1  et  f1  (Cout =  3 )
Mariage entre  h2  et  f4  (Cout =  0 )
Mariage entre  h3  et  f2  (Cout =  0 )
Mariage entre  h4  et  f3  (Cout =  2 )
